# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

,0
payment_history,float64
location_parameter,float64
stem_degree_score,float64
gpa_ranking,float64
alumni_success,float64
study_major_code,float64
time_to_completion,float64
finance_workshop_score,float64
cohort_ranking,float64
total_loan_score,float64


In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

,count
credit_ranking,
1,855
0,744


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Define the features dataset X by dropping the "credit_ranking" column from the original DataFrame
X = loans_df.drop("credit_ranking", axis=1)

# Display a sample of y
print(y.head())


0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64


In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
print(X.head())


   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0              7.4                0.70               0.00          1.9   
1              7.8                0.88               0.00          2.6   
2              7.8                0.76               0.04          2.3   
3             11.2                0.28               0.56          1.9   
4              7.4                0.70               0.00          1.9   

   alumni_success  study_major_code  time_to_completion  \
0           0.076              11.0                34.0   
1           0.098              25.0                67.0   
2           0.092              15.0                54.0   
3           0.075              17.0                60.0   
4           0.076              11.0                34.0   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                  0.9978            3.51              0.56   
1                  0.9968            3.20              0.68   
2          

### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Display the shape of the training and testing datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1279, 11)
X_test shape: (320, 11)
y_train shape: (1279,)
y_test shape: (320,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Transform the features training dataset
X_train_scaled = scaler.transform(X_train)

# Transform the features testing dataset
X_test_scaled = scaler.transform(X_test)

# Display the scaled features training dataset
print("Scaled X_train:")
print(X_train_scaled)

# Display the scaled features testing dataset
print("Scaled X_test:")
print(X_test_scaled)


Scaled X_train:
[[-0.73307913  0.6648928  -1.25704443 ...  0.98846046  0.0630946
  -0.87223395]
 [ 1.06774091 -0.62346154  1.52314768 ... -1.7535127  -0.17390392
  -0.77978452]
 [-1.74604041 -1.07158479 -1.35814232 ...  2.32756363  0.77409018
   3.28799021]
 ...
 [-0.95818164  1.08500835 -0.90320179 ...  0.22325865 -1.00339876
   0.32960859]
 [-0.62052788  0.55286199 -1.35814232 ...  0.35079228 -0.47015208
  -1.33448108]
 [ 0.44870902 -0.73549236  1.16930505 ... -0.6694768   0.18159387
   1.90124882]]
Scaled X_test:
[[ 0.27988214 -0.67947695  1.87699031 ...  1.43482818  0.00384497
   0.05226031]
 [ 0.22360652  0.55286199  0.05722821 ... -0.35064271 -0.17390392
  -0.22508797]
 [ 1.18029216 -1.07158479  1.57369663 ... -0.79701044  0.2408435
   0.79185571]
 ...
 [-0.67680351  0.77692361 -1.35814232 ...  0.79716001 -0.7071506
  -0.96468337]
 [-1.18328414 -0.79150776  0.81546242 ...  0.86092682  1.24808723
   0.69940629]
 [ 1.4616703  -1.18361561  1.27040294 ... -0.22310908  0.65559092
   0

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the number of inputs (features) to the model
n_features = X_train_scaled.shape[1]

# Review the number of features
print("Number of features:", n_features)


Number of features: 11


In [11]:
# Define the number of hidden nodes for the first hidden layer
n_hidden1 = 64

# Define the number of hidden nodes for the second hidden layer
n_hidden2 = 32

# Define the number of neurons in the output layer
n_output = 1  # Assuming this is a binary classification task, adjust if necessary


In [12]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(n_hidden1, input_dim=n_features, activation='relu'))

# Add the second hidden layer
model.add(Dense(n_hidden2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(n_output, activation='sigmoid'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Display the Sequential model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5308 - loss: 0.6940
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7417 - loss: 0.5503
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7430 - loss: 0.5353
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7469 - loss: 0.5223
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7552 - loss: 0.5085
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7428 - loss: 0.5103 
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7773 - loss: 0.4749
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7711 - loss: 0.4666
Epoch 9/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7546 - loss: 0.4821
Epoch 10/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7802 - loss: 0.4591
Epoch 11/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7695 - loss: 0.4628
Epoch 12/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7787 - l

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [16]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print("Model Loss:", loss)
print("Model Accuracy:", accuracy)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7993 - loss: 0.5299  
Model Loss: 0.572325587272644
Model Accuracy: 0.7562500238418579


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [17]:
# Set the model's file path
model_file_path = "student_loans.keras"

# Export your model to a keras file
model.save(model_file_path)
print("Model saved successfully as", model_file_path)

Model saved successfully as student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [18]:
# Set the model's file path
model_file_path = "student_loans.keras"

# Load the model to a new object
loaded_model = tf.keras.models.load_model(model_file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [19]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Create a DataFrame to store the predictions
predictions_df = pd.DataFrame(predictions, columns=['predicted_credit_ranking'])

# Display a sample of the predictions
print(predictions_df.head())


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  
   predicted_credit_ranking
0                  0.156838
1                  0.502040
2                  0.801374
3                  0.717700
4                  0.994796


In [20]:
# Round the predictions to binary results (0 or 1)
rounded_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Create a DataFrame to store the rounded predictions
rounded_predictions_df = pd.DataFrame(rounded_predictions, columns=['rounded_predicted_credit_ranking'])

# Display a sample of the rounded predictions
print(rounded_predictions_df.head())


   rounded_predicted_credit_ranking
0                                 0
1                                 1
2                                 1
3                                 1
4                                 1


### Step 4: Display a classification report with the y test data and predictions

In [22]:
# Print the classification report with the y test data and predictions
from sklearn.metrics import classification_report
import numpy as np

# Convert the rounded predictions to a numpy array
rounded_predictions_np = np.array(rounded_predictions)

# Print the classification report
print(classification_report(y_test, rounded_predictions_np))


              precision    recall  f1-score   support

           0       0.73      0.78      0.75       154
           1       0.78      0.73      0.76       166

    accuracy                           0.76       320
   macro avg       0.76      0.76      0.76       320
weighted avg       0.76      0.76      0.76       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.
Answer:

  1.1 Student Profile Data: This would include information such as the student's academic performance, major, career goals, financial situation, and any relevant background information. This data would help in understanding the student's needs and preferences when it comes to selecting a student loan.

  1.2 Loan Product Data: This would include details about various student loan products available, such as interest rates, repayment terms, eligibility criteria, and any special features or benefits. This data would help in matching the student's profile with the most suitable loan options.

  1.3 Loan Repayment Data: This would include information on loan repayment history, default rates, and overall loan performance. This data would help in predicting the likelihood of a student successfully repaying a loan and selecting appropriate loan options accordingly.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

Answer: Based on the data used in this recommendation system, the model would be using content-based filtering. Content-based filtering recommends items based on the attributes of the items and a profile of the user's preferences. In this case, the model is recommending student loan options based on the student's profile data and loan product data, without relying on the preferences or behavior of other users (collaborative filtering) or contextual information (context-based filtering).

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

Answer: Two real-world challenges to consider while building a recommendation system for student loans are:

3.1 Data Privacy and Security: Student loan data contains sensitive information about individuals' financial situations and academic performance. Ensuring the privacy and security of this data is crucial to maintaining trust with users. Implementing robust data protection measures and compliance with data privacy regulations is essential to address this challenge.

3.2 Bias and Fairness: Building a recommendation system that is fair and unbiased is important to ensure equal opportunities for all students. Biases in the data used to train the model, such as historical loan approval rates or demographic information, can lead to unfair recommendations. Addressing bias in the data and ensuring fairness in the recommendation process is critical to avoid discrimination and provide equitable loan options for all students.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

